In [ ]:
# Importieren der benötigten Bibliotheken

# Datenmanipulation
import pandas as pd
import re

# Visualisierung
import seaborn as sns
import matplotlib.pyplot as plt

# Eigene Module
from dpp.data import load_from_kaggle

# Preparation


**Gather Data**


In [ ]:
# Daten von Kaggle herunterladen
dataset_link = "thedevastator/improving-naval-vessel-condition-through-machine" # replace with your dataset link from Kaggle 
destination = "../data/raw"
dataset_name = dataset_link.split("/")[-1]

files = load_from_kaggle(
    dataset_link=dataset_link, 
    destination=destination,
    )

In [ ]:
# Anzeigen der heruntergeladenen Daten
files

In [ ]:
# Einlesen der Daten
df = pd.read_csv("/".join([destination, dataset_name, files[0]]), index_col=0)
df.head()

# Explorative Datenanalayse (EDA)


**Understand Data**


In [ ]:
# Erste Übersicht über die Daten
display(
    "Shape",
    df.shape,
    "Description",
    df.describe().round(2),
    "Duplicates",
    df.duplicated().sum(),
)

# Übersicht über die Spalten
pd.DataFrame(
    {
        "Data Types": df.dtypes,
        "Missing Values": df.isnull().sum(),
        "Unique Values": df.nunique(),
        "Sample Values": [df[col].sample(3).tolist() for col in df.columns]
    })

**Rename Columns**


In [ ]:
# Bereinigen der Spaltennamen
def clean_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = col
        new_col = re.sub(r"\s+", "_", new_col) # Unterstriche ersetzen Leerzeichen
        new_col = new_col.strip("_") # Führende/nachfolgende Unterstriche entfernen
        new_cols.append(new_col)
    df.columns = new_cols
    return df

df = clean_col_names(df)

df.head()

In [ ]:
# Entfernen irrelevanter Spalten gleicher Wert in allen Zeilen
cols_to_drop = [col for col in df.columns if df[col].nunique() <= 1]
print(f"Folgende Spalten werden entfernt: {cols_to_drop}")
df = df.drop(columns=cols_to_drop)

**Outliers Detection**


In [ ]:
# Verteilung und Boxplots 
for col in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    sns.histplot(x=df[col], kde=True, ax=axes[0], color="#009292")
    sns.boxplot(x=df[col], ax=axes[1], color="#009292")
    axes[0].set_title(f"Histogram of {col}")
    axes[1].set_title(f"Boxplot of {col}")
    plt.tight_layout()

**Check Correlations**


In [ ]:
# Korrelationen visualisieren
plt.figure(figsize=(16, 12))
sns.heatmap(df.corr(numeric_only=True), annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Correlation Matrix")

In [ ]:
# Visualisierung der bivariaten Beziehungen
sns.pairplot(df, plot_kws={"color": "#009292"}, diag_kws={"color": "#074650"})

In [ ]:
# Speichern der bereinigten Daten
destination_path = "../data/processed"
df.to_csv(f"{destination_path}/cleaned_data.csv", index=False)

**Zustandsverteilung**

In [ ]:
# Schwellenwerte für Zerfallzustand
thresholds = {
    "compressor": {"good": 0.98},
    "turbine": {"good": 0.99}
}

# Definition der Zustände und zugehörigen Farben
state_order = ["Gut", "Handlungsbedarf"]
state_colors = {"Gut": "#009292", "Handlungsbedarf": "#074650"}

# Funktion zur Umwandlung in zwei Zustände
def get_binary_state(value, component_type):
    th = thresholds[component_type]
    return "Gut" if value >= th["good"] else "Handlungsbedarf"

# Zu visualisierende Komponenten
components_to_plot = [
    {"col": "GT_Compressor_decay_state_coefficient", "type": "compressor", "title": "Zustandsverteilung Kompressor"},
    {"col": "GT_Turbine_decay_state_coefficient", "type": "turbine", "title": "Zustandsverteilung Turbine"}
]

# Erstellen der Visualisierung
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

for i, component in enumerate(components_to_plot):
    # Temporäre Spalte mit den Zuständen erstellen
    state_series = df[component["col"]].apply(lambda x: get_binary_state(x, component["type"]))
    
    # Daten für das Diagramm vorbereiten
    counts = state_series.value_counts().reindex(state_order).fillna(0)
    colors = [state_colors[state] for state in counts.index]
    
    # Kuchendiagramm zeichnen
    axes[i].pie(
        counts,
        labels=counts.index,
        autopct="%1.1f%%",
        startangle=90,
        colors=colors,
        textprops={"fontsize": 12}
    )
    axes[i].set_title(component["title"], fontsize=14)

plt.tight_layout()
plt.show()